# Imports

In [3]:
import tensorflow as tf

# Dataset
import numpy as np
import pandas as pd

# Utils
from datetime import datetime


# Constants

In [4]:
# Model
learning_rate = 0.02
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
loss = tf.keras.losses.Huber()
metrics = ["mae"]
epochs = 1000

# File
dataset_file = './dataset/user_1/user_1_data.csv'
model_1_file = './model/user_1/'
model_2_file = './model/...'

# Data

In [28]:
data = pd.read_csv(dataset_file)

# Converting date string to datetime
data["date"] = pd.to_datetime(data["date"])
data["day_of_week"] = data["date"].dt.day_of_week
data["month"] = data["date"].dt.month
data["year"] = data["date"].dt.year
del data["date"
# Converting time to cumulative minute
# source: https://stackoverflow.com/questions/17951820/convert-hhmmss-to-minutes-using-python-pandas
# credit: Andy Hayden
data["time"] = data["time"].str.split(':').apply(lambda time: int(time[0]) * 60 + int(time[1]))

data
# max(data["date"])

,date,time,latitude,longitude,day_of_week,month,year
0,2022-03-07,0,-6.268917,106.780112,0,3,2022
1,2022-03-07,1,-6.268917,106.780112,0,3,2022
2,2022-03-07,2,-6.268917,106.780112,0,3,2022
3,2022-03-07,3,-6.268917,106.780112,0,3,2022
4,2022-03-07,4,-6.268917,106.780112,0,3,2022
...,...,...,...,...,...,...,...
20155,2022-03-20,1435,-6.268917,106.779552,6,3,2022
20156,2022-03-20,1436,-6.268917,106.779552,6,3,2022
20157,2022-03-20,1437,-6.268917,106.779552,6,3,2022
20158,2022-03-20,1438,-6.268917,106.779552,6,3,2022


# Model

In [5]:
def create_model():
    """Create Forecasting Model
    Model used: LSTM
    output should consist of 3 item, [datetime, latitude, longitude]
    """
    # Generating model
    model = tf.keras.models.Sequential([
        tf.keras.layers.LSTM(24),
        tf.keras.layers.Dense(3, activation='sigmoid'),
        tf.keras.layers.Dense(3, activation='linear')
    ])
    
    # Compiling model
    model.compile(
        loss=loss,
        optimizer=optimizer,
        metrics=metrics,
    )
    
    # Train model
    # model.fit(data, epochs=epochs)
    
    return model
